In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [67]:
import re
import pandas as pd
import numpy as np


import plotly.express as px
import plotly.graph_objects as go
import xgboost as xgb
from scipy.stats import skew, kurtosis

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score



In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

1. Load the Data
First, we need to load the dataset into a suitable environment for analysis. This usually involves using a tool or library to read the data into a DataFrame for easier manipulation.

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Data_Analysis_Projects/my_data_analysis_project/data/raw/laptops_data.csv")

In [5]:
data = df.copy()

2.Understand the Structure and Content

a. View the Data:

Display the first few rows to get a sense of the data.
Check the column names to understand what information each column holds.

b. Summary Statistics:

Generate summary statistics for numerical columns (mean, median, standard deviation, etc.).
Generate frequency distributions for categorical columns.

In [6]:
# Display the first few rows of the dataset
data.head()

,Name,Price,Spec Score,Processor,Clock-speed,Operating System,SSD Capacity,RAM Type,Graphics Processor,Display Size,Display Resolution,Capacity,Aspect Ratio,Overall Rating,Battery Cell,Battery type,Power Supply,Weight,Touchscreen,Colour(s),Display Features
0,Acer One 14 Z8-415 (UN.599SI.020) Laptop (Core...,"34,990",64.0,Intel Core i5-1155G7 (11th Gen),NaN,Windows 11 Home Basic,512 GB,DDR4,Intel Iris Xe,14 Inches (35.56 cm),1920 x 1080 Pixels,16 GB,16:9,3.7,2 Cell,Li-Ion,45 W,1.49 Kg weight (Light-weight),No,Pure Silver,Full HD 1920 x 1080 Resolution Antiglare LED B...
1,HP 15-fc0026AU (7L030PA) Laptop (AMD Quad Core...,"36,990",NaN,AMD Quad Core Ryzen 3 - 7320U,2.4 Ghz,Windows 11 Home Basic,512 GB,LPDDR5,AMD Radeon,15.6 Inches (39.62 cm),1920 x 1080 Pixels,8 GB,16:9,NaN,3 Cell,Li-Ion,NaN,1.75 Kg weight,No,Natural Silver,FHD\nMicro-Edge\nAnti-Glare\nBrightness: 250 N...
2,HP 15s-fq5007TU (67V50PA) Laptop (Core i3 12th...,"37,166",66.0,Intel Core i3-1215U (12th Gen),4.2 Ghz,Windows 11 Home Basic,512 GB,DDR4,Intel UHD,15.6 Inches (39.62 cm),1920 x 1080 Pixels,8 GB,NaN,4.2,3 Cell,Li-Ion,65 W,1.69 Kg weight,No,Natural Silver,Diagonal\nFHD (1920 x 1080)\nIPS\nMicro-Edge\n...
3,Acer Predator Helios Neo 16 PHN16-71 (NH.QLTSI...,"112,980",61.0,Intel Core i7-13700HX (13th Gen),NaN,Windows 11 Home Basic,1 TB,DDR5,NVIDIA GeForce RTX 4050,16 Inches (40.64 cm),1920 x 1200 Pixels,16 GB,NaN,4.4,4 Cell,Li-Ion,230 W,2.6 Kg weight,No,Black,IPS Display With WUXGA 1920 x 1200\nHigh-Brigh...
4,Asus TUF Gaming F17 FX706HF-HX018W Laptop (Cor...,"57,990",65.0,Intel Core i5-11400H (11th Gen),2.7 Ghz,Windows 11 Home Basic,512 GB,DDR4,NVIDIA GeForce RTX 2050,17.3 Inches (43.94 cm),1920 x 1080 Pixels,8 GB,16:9,4.3,3 Cell,Li-Ion,150 W,2.60 Kg weight,No,Graphite Black,FHD (1920 x 1080) 16:9\nValue IPS-level\nAnti-...


In [7]:
# Summary statistics for numerical columns
data.describe()

,Spec Score,Overall Rating
count,2573.000000,1968.000000
mean,68.188496,4.058028
std,8.086324,0.637816
min,34.000000,1.000000
25%,64.000000,3.900000
50%,67.000000,4.200000
75%,71.000000,4.300000
max,96.000000,5.000000


In [8]:
# Checking for duplicate values
data.duplicated().sum()

0

In [9]:
# Checking for missing values
data.isnull().sum()

Name                     0
Price                    0
Spec Score              96
Processor                3
Clock-speed            162
Operating System         3
SSD Capacity           142
RAM Type                80
Graphics Processor      38
Display Size             4
Display Resolution      20
Capacity                 3
Aspect Ratio          2140
Overall Rating         701
Battery Cell           496
Battery type           143
Power Supply           543
Weight                  58
Touchscreen             45
Colour(s)               24
Display Features        74
dtype: int64

In [10]:
# Display the data types of each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2669 entries, 0 to 2668
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                2669 non-null   object 
 1   Price               2669 non-null   object 
 2   Spec Score          2573 non-null   float64
 3   Processor           2666 non-null   object 
 4   Clock-speed         2507 non-null   object 
 5   Operating System    2666 non-null   object 
 6   SSD Capacity        2527 non-null   object 
 7   RAM Type            2589 non-null   object 
 8   Graphics Processor  2631 non-null   object 
 9   Display Size        2665 non-null   object 
 10  Display Resolution  2649 non-null   object 
 11  Capacity            2666 non-null   object 
 12  Aspect Ratio        529 non-null    object 
 13  Overall Rating      1968 non-null   float64
 14  Battery Cell        2173 non-null   object 
 15  Battery type        2526 non-null   object 
 16  Power 

Data Cleaning: Price Column

Steps:

Remove Commas: Convert the price values from strings to integers by removing commas.

Check for Missing Values: Ensure there are no missing values after conversion.

Convert Data Type: Ensure the Price column is of the appropriate numeric data type (float).

In [11]:
# Remove commas and convert the Price column to numeric values
data['Price'] = data['Price'].str.replace(',', '').astype(float)

In [12]:
data['Price'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2669 entries, 0 to 2668
Series name: Price
Non-Null Count  Dtype  
--------------  -----  
2669 non-null   float64
dtypes: float64(1)
memory usage: 21.0 KB


In [13]:
data.sample()

,Name,Price,Spec Score,Processor,Clock-speed,Operating System,SSD Capacity,RAM Type,Graphics Processor,Display Size,Display Resolution,Capacity,Aspect Ratio,Overall Rating,Battery Cell,Battery type,Power Supply,Weight,Touchscreen,Colour(s),Display Features
754,Dell Vostro 14 3400 (D552217WIN9D) Laptop (Cor...,39275.0,NaN,Intel Core i3-1115G4 (11th Gen),4.0 Ghz,Windows 11 Home Basic,256 GB,NaN,Intel Integrated UHD,14 Inches (35.56 cm),1920 x 1080 Pixels,8 GB,NaN,NaN,3 Cell,Li-Ion,65 W,1.59 Kg weight,No,Dune,Full HD WVA AG Narrow Border


To perform a visual exploration of the Price column, we will:

Plot a Histogram: To visualize the distribution of prices.

Plot a Boxplot: To identify outliers and understand the spread of prices.

Calculate Skewness and Kurtosis: To understand the asymmetry and tailedness of the price distribution.

In [14]:
# Plotly histogram for Price distribution
fig_hist = px.histogram(data, x='Price', nbins=30, title='Price Distribution', marginal="box")
fig_hist.update_layout(xaxis_title='Price', yaxis_title='Frequency')
fig_hist.show()

# Plotly boxplot for Price
fig_box = go.Figure()
fig_box.add_trace(go.Box(y=data['Price'], name='Price', boxmean=True))
fig_box.update_layout(title='Boxplot of Price', yaxis_title='Price')
fig_box.show()

# Calculate skewness and kurtosis
price_skewness = skew(data['Price'])
price_kurtosis = kurtosis(data['Price'])

print(f'Skewness: {price_skewness}')
print(f'Kurtosis: {price_kurtosis}')


Skewness: 2.3605869451555743
Kurtosis: 7.140159894822064


Incorporating Outlier Flags

Incorporating Outlier Flags involves adding an additional column to the dataset to mark outliers. This flag can be used in visualizations and analyses to highlight and handle outliers without modifying the actual data values.

Example:

Adding a column Outlier_Flag where a value of 1 indicates an outlier and 0 indicates a non-outlier.
Advantages:

Maintains the original data values.
Allows for flexible analysis, as outliers can be included or excluded dynamically based on the flag.

Disadvantages:

Requires a method to define what constitutes an outlier.
Adds an extra step in data preprocessing.

In [15]:
# Identify thresholds for outliers
Q1 = data['Price'].quantile(0.25)
Q3 = data['Price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Add outlier flag
data['Outlier_Flag'] = ((data['Price'] < lower_bound) | (data['Price'] > upper_bound)).astype(int)


 Stratified Analysis:

In [16]:
data['Outlier_Flag'].value_counts()

Outlier_Flag
0    2484
1     185
Name: count, dtype: int64

In [17]:
# Plotly boxplot for Price with Outlier differentiation
fig_box = px.box(data, x='Outlier_Flag', y='Price', title='Boxplot of Price with Outliers Highlighted')
fig_box.update_layout(xaxis_title='Outlier Status', yaxis_title='Price')
fig_box.show()

# Plotly scatter plot for Price with Outlier differentiation
fig_scatter = px.scatter(data, x=data.index, y='Price', color='Outlier_Flag', title='Scatter Plot of Price with Outliers Highlighted')
fig_scatter.update_layout(xaxis_title='Index', yaxis_title='Price')
fig_scatter.show()



In [18]:
data[data['Outlier_Flag']==1].describe()

,Price,Spec Score,Overall Rating,Outlier_Flag
count,185.000000,175.000000,103.000000,185.0
mean,252072.108108,71.445714,3.967961,1.0
std,65118.865031,8.582672,1.132775,0.0
min,174990.000000,58.000000,1.000000,1.0
25%,199900.000000,66.500000,3.500000,1.0
50%,239900.000000,69.000000,4.300000,1.0
75%,289990.000000,74.000000,5.000000,1.0
max,489883.000000,96.000000,5.000000,1.0


Define Price Ranges:

Segment the data into different price ranges (e.g., low, medium, high).


In [19]:
# Define price ranges
bins = [0, 30000, 60000, 90000, 120000, np.inf]
labels = ['Low', 'Medium-Low', 'Medium', 'Medium-High', 'High']

# Create a new column for price ranges after the Price column
data.insert(data.columns.get_loc('Price') + 1, 'Price_Range', pd.cut(data['Price'], bins=bins, labels=labels))

# Verify the new column
data[['Price', 'Price_Range']].head()


,Price,Price_Range
0,34990.0,Medium-Low
1,36990.0,Medium-Low
2,37166.0,Medium-Low
3,112980.0,Medium-High
4,57990.0,Medium-Low


In [20]:
data['Price_Range'].value_counts()

Price_Range
Medium-Low     1026
Medium          750
High            440
Medium-High     292
Low             161
Name: count, dtype: int64

Create a Brand Column:

Extract the brand information from the Name column.

In [21]:
# Extract brand information from the Name column
data['Brand'] = data['Name'].apply(lambda x: x.split()[0])

# Insert the Brand column after the Name column
data.insert(data.columns.get_loc('Name') + 1, 'Brand', data.pop('Brand'))

# Verify the new column
data[['Name', 'Brand']].head()


,Name,Brand
0,Acer One 14 Z8-415 (UN.599SI.020) Laptop (Core...,Acer
1,HP 15-fc0026AU (7L030PA) Laptop (AMD Quad Core...,HP
2,HP 15s-fq5007TU (67V50PA) Laptop (Core i3 12th...,HP
3,Acer Predator Helios Neo 16 PHN16-71 (NH.QLTSI...,Acer
4,Asus TUF Gaming F17 FX706HF-HX018W Laptop (Cor...,Asus


In [22]:
data['Brand'].value_counts()

Brand
Asus         685
Lenovo       558
HP           508
Dell         292
MSI          230
Acer         122
Apple         62
Samsung       53
Infinix       39
Microsoft     24
Honor         14
Gigabyte      13
Fujitsu       13
LG            11
Avita         11
Zebronics      8
Xiaomi         6
Wings          4
Tecno          3
Coconics       3
Primebook      2
Razer          2
Lava           1
VAIO           1
realme         1
Reliance       1
Smartron       1
Huawei         1
Name: count, dtype: int64

Next Steps: Stratified Analysis by Brand and Price Range

Perform Analysis within Each Brand and Price Range:

Generate summary statistics for each combination of brand and price range.

In [23]:
# Perform analysis within each brand and price range
stratified_summary = data.groupby(['Brand', 'Price_Range'])['Price'].describe()
print(stratified_summary)


                       count           mean           std       min  \
Brand     Price_Range                                                 
Acer      Low           13.0   24575.384615   4460.248714   16990.0   
          Medium-Low    67.0   44120.253731   9343.438001   30360.0   
          Medium        22.0   74790.363636   9803.075259   61499.0   
          Medium-High   11.0  101991.545455   5457.254958   92990.0   
          High           9.0  165442.888889  38387.668379  129990.0   
Apple     Medium         4.0   76240.000000   9604.686356   67990.0   
          Medium-High    9.0  109441.111111   9983.468140   90900.0   
          High          49.0  218111.040816  78375.261040  129900.0   
Asus      Low           36.0   23343.416667   4681.923691   13990.0   
          Medium-Low   246.0   46563.808943   9081.788676   30690.0   
          Medium       199.0   73773.286432   8710.889330   60199.0   
          Medium-High   79.0  104605.012658   8688.150885   90519.0   
      

In [26]:

# Calculate the count of laptops for each brand and price range
count_prices = data.groupby(['Brand', 'Price_Range'])['Price'].count().unstack()

# Create a heatmap using Plotly
fig_heatmap = go.Figure(data=go.Heatmap(
    z=count_prices.values,
    x=count_prices.columns,
    y=count_prices.index,
    colorscale='YlGnBu',
    showscale=True,
    hoverongaps=False))

fig_heatmap.update_layout(title='Concentration of Laptops by Brand and Price Range', xaxis_title='Price Range', yaxis_title='Brand')
fig_heatmap.show()


In [27]:
# Function to extract model name
def extract_model(name, brand):
    return name.replace(brand, '').split('Laptop')[0].strip()

# Extract model name and create Model_Name column
data['Model_Name'] = data.apply(lambda x: extract_model(x['Name'], x['Brand']), axis=1)

# Insert the Model_Name column after the Brand column
data.insert(data.columns.get_loc('Brand') + 1, 'Model_Name', data.pop('Model_Name'))

# Verify the new column
data[['Name', 'Brand', 'Model_Name']].head(10)


,Name,Brand,Model_Name
0,Acer One 14 Z8-415 (UN.599SI.020) Laptop (Core...,Acer,One 14 Z8-415 (UN.599SI.020)
1,HP 15-fc0026AU (7L030PA) Laptop (AMD Quad Core...,HP,15-fc0026AU (7L030PA)
2,HP 15s-fq5007TU (67V50PA) Laptop (Core i3 12th...,HP,15s-fq5007TU (67V50PA)
3,Acer Predator Helios Neo 16 PHN16-71 (NH.QLTSI...,Acer,Predator Helios Neo 16 PHN16-71 (NH.QLTSI.006)
4,Asus TUF Gaming F17 FX706HF-HX018W Laptop (Cor...,Asus,TUF Gaming F17 FX706HF-HX018W
5,HP 15s-fq5112TU (6Q2M3PA) Laptop (Core i5 12th...,HP,15s-fq5112TU (6Q2M3PA)
6,Asus TUF Gaming F15 FX506HF-HN076W Laptop (Cor...,Asus,TUF Gaming F15 FX506HF-HN076W
7,HP 15s-fq5330TU (9D3P0PA) Laptop (Core i5 12th...,HP,15s-fq5330TU (9D3P0PA)
8,Samsung Galaxy Book 2 NP750XED-KC1IN 15.6 Lapt...,Samsung,Galaxy Book 2 NP750XED-KC1IN 15.6
9,Asus VivoBook 15 X1502ZA-EJ993WS Laptop (Core ...,Asus,VivoBook 15 X1502ZA-EJ993WS


In [28]:

# Function to clean the model name
def clean_model_name(model_name):
    # Remove text within parentheses
    model_name = re.sub(r'\(.*?\)', '', model_name)
    # Remove hyphenated suffixes
    model_name = re.sub(r'-[A-Za-z0-9]+$', '', model_name)
    return model_name.strip()

# Clean the Model_Name column
data['Model_Name'] = data['Model_Name'].apply(clean_model_name)

# Verify the cleaned column
cleaned_model_name_sample = data[['Name', 'Brand', 'Model_Name']].head(10)
cleaned_model_name_sample


,Name,Brand,Model_Name
0,Acer One 14 Z8-415 (UN.599SI.020) Laptop (Core...,Acer,One 14 Z8-415
1,HP 15-fc0026AU (7L030PA) Laptop (AMD Quad Core...,HP,15-fc0026AU
2,HP 15s-fq5007TU (67V50PA) Laptop (Core i3 12th...,HP,15s-fq5007TU
3,Acer Predator Helios Neo 16 PHN16-71 (NH.QLTSI...,Acer,Predator Helios Neo 16 PHN16-71
4,Asus TUF Gaming F17 FX706HF-HX018W Laptop (Cor...,Asus,TUF Gaming F17 FX706HF
5,HP 15s-fq5112TU (6Q2M3PA) Laptop (Core i5 12th...,HP,15s-fq5112TU
6,Asus TUF Gaming F15 FX506HF-HN076W Laptop (Cor...,Asus,TUF Gaming F15 FX506HF
7,HP 15s-fq5330TU (9D3P0PA) Laptop (Core i5 12th...,HP,15s-fq5330TU
8,Samsung Galaxy Book 2 NP750XED-KC1IN 15.6 Lapt...,Samsung,Galaxy Book 2 NP750XED-KC1IN 15.6
9,Asus VivoBook 15 X1502ZA-EJ993WS Laptop (Core ...,Asus,VivoBook 15 X1502ZA


In [29]:
data.sample()

,Name,Brand,Model_Name,Price,Price_Range,Spec Score,Processor,Clock-speed,Operating System,SSD Capacity,RAM Type,Graphics Processor,Display Size,Display Resolution,Capacity,Aspect Ratio,Overall Rating,Battery Cell,Battery type,Power Supply,Weight,Touchscreen,Colour(s),Display Features,Outlier_Flag
280,HP 14s-fq1029AU (3V6P4PA) Laptop (AMD Quad Cor...,HP,14s-fq1029AU,43999.0,Medium-Low,63.0,AMD Quad Core Ryzen 3 5300U,3.4 Ghz,Windows 10 Home Basic,512 GB,DDR4,AMD Radeon,14 Inches (35.56 cm),1920 x 1080 Pixels,8 GB,NaN,4.5,3 Cell,Li-Ion,65 W AC Adapter W,1.47 Kg weight (Light-weight),No,1.46,Full HD Anti-glare Display,0


In [30]:
data[data['Processor'].isnull()]

,Name,Brand,Model_Name,Price,Price_Range,Spec Score,Processor,Clock-speed,Operating System,SSD Capacity,RAM Type,Graphics Processor,Display Size,Display Resolution,Capacity,Aspect Ratio,Overall Rating,Battery Cell,Battery type,Power Supply,Weight,Touchscreen,Colour(s),Display Features,Outlier_Flag
620,Lenovo Ideapad Gaming 3 15IHU6 (82K1019AIN) La...,Lenovo,Ideapad Gaming 3 15IHU6,69990.0,Medium,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
621,Dell Vostro 3401 (D552226WIN9BE) Laptop (Core ...,Dell,Vostro 3401,42290.0,Medium-Low,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1677,Dell Inspiron 15 3511 (D560743WIN9B) Laptop (C...,Dell,Inspiron 15 3511,38499.0,Medium-Low,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [31]:
# Update the specific rows with missing processor information
data.loc[data['Name'].str.contains("Lenovo Ideapad Gaming 3 15IHU6"), 'Processor'] = "Intel Core i5-11300H (11th Gen)"
data.loc[data['Name'].str.contains("Dell Vostro 3401"), 'Processor'] = "Intel Core i3-1005G1 (10th Gen)"
data.loc[data['Name'].str.contains("Dell Inspiron 15 3511"), 'Processor'] = "Intel Core i3-1115G4 (11th Gen)"

# Verify the changes
data[['Name', 'Processor']].loc[data['Processor'].isnull()]


,Name,Processor


In [32]:
data['Processor'].isnull().sum()

0

In [33]:
# Update the specific rows with the base clock speed information
data.loc[data['Name'].str.contains("Lenovo Ideapad Gaming 3 15IHU6"), 'Clock-speed'] = "3.1 GHz"
data.loc[data['Name'].str.contains("Dell Vostro 3401"), 'Clock-speed'] = "1.2 GHz"
data.loc[data['Name'].str.contains("Dell Inspiron 15 3511"), 'Clock-speed'] = "3.0 GHz"

In [34]:
# Update the specific rows with the operating system information
data.loc[data['Name'].str.contains("Lenovo Ideapad Gaming 3 15IHU6"), 'Operating System'] = "Windows 11 Home"
data.loc[data['Name'].str.contains("Dell Vostro 3401"), 'Operating System'] = "Windows 10 Home"
data.loc[data['Name'].str.contains("Dell Inspiron 15 3511"), 'Operating System'] = "Windows 10 Home"


In [35]:
data['Processor'].isnull().sum()

0

In [36]:
data.sample()

,Name,Brand,Model_Name,Price,Price_Range,Spec Score,Processor,Clock-speed,Operating System,SSD Capacity,RAM Type,Graphics Processor,Display Size,Display Resolution,Capacity,Aspect Ratio,Overall Rating,Battery Cell,Battery type,Power Supply,Weight,Touchscreen,Colour(s),Display Features,Outlier_Flag
1840,Dell Alienware x15 R2 (D569941WIN9) Laptop (Co...,Dell,Alienware x15 R2,285590.0,High,65.0,Intel Core i7-12700H (12th Gen),3.4 Ghz,Windows 11 Home Basic,1 TB,LPDDR5,NVIDIA GeForce RTX 3070 Ti,15.6 Inches (39.62 cm),1920 x 1080 Pixels,32 GB,NaN,5.0,6 Cell,Li-Ion,65 W,2.34 Kg weight,No,Lunar Light,FHD IPS,1


Overview

This document outlines the method for extracting detailed information from the 'Processor' column of a laptop dataset and splitting this information into several new columns to facilitate easier data analysis and improve data quality.


Process Description
Column Extraction Objective:

Processor_Brand: Identifies the brand of the processor (e.g., Intel, AMD).

Core Configuration: Describes the core configuration if mentioned (e.g., Dual Core, Quad Core).

Series: Indicates the series or model of the processor (e.g., Ryzen 5, Core i7).

Generation: Specifies the generation of the processor if available (e.g., 11th Gen, 12th Gen).

Methods Used:

Regular expressions are employed to extract specific patterns from the 'Processor' column.

Conditional checks are used to handle variations and ensure that missing data is labeled as 'Unknown'.

In [37]:
laptop_data = data.copy()

In [38]:

# Define functions to extract the processor details
def extract_processor_brand(processor):
    brands = ["Intel", "AMD", "MediaTek", "Apple"]
    processor_lower = processor.lower()
    for brand in brands:
        if brand.lower() in processor_lower:
            return brand
    return "Unknown"

def extract_core_config(processor):
    core_config_match = re.search(r'(Quad Core|Hexa Core|Octa Core|Dual Core)', processor, re.IGNORECASE)
    return core_config_match.group(0) if core_config_match else "Unknown"

def extract_series(processor):
    series_match = re.search(r'(Ryzen \d+|Core i\d+|Apple M\d+|MT\d+|Pentium Gold|Core Ultra \d+)', processor, re.IGNORECASE)
    return series_match.group(0) if series_match else "Unknown"

def extract_generation(processor):
    gen_match = re.search(r'(\d+th Gen|\d+rd Gen|\d+nd Gen|\d+st Gen)', processor, re.IGNORECASE)
    return gen_match.group(0) if gen_match else "Unknown"

# Apply the functions to extract data
data['Processor_Brand'] = data['Processor'].apply(extract_processor_brand)
data['Core Configuration'] = data['Processor'].apply(extract_core_config)
data['Series'] = data['Processor'].apply(extract_series)
data['Generation'] = data['Processor'].apply(extract_generation)

# Rearrange columns to insert the new ones right after 'Processor'
new_columns = ['Processor', 'Processor_Brand', 'Core Configuration', 'Series', 'Generation']
existing_columns = [col for col in data.columns if col not in new_columns]
final_columns_order = new_columns + existing_columns
laptops_data_new = data[final_columns_order]

# Display the first few rows to confirm the setup
laptops_data_new.head()


,Processor,Processor_Brand,Core Configuration,Series,Generation,Name,Brand,Model_Name,Price,Price_Range,Spec Score,Clock-speed,Operating System,SSD Capacity,RAM Type,Graphics Processor,Display Size,Display Resolution,Capacity,Aspect Ratio,Overall Rating,Battery Cell,Battery type,Power Supply,Weight,Touchscreen,Colour(s),Display Features,Outlier_Flag
0,Intel Core i5-1155G7 (11th Gen),Intel,Unknown,Core i5,11th Gen,Acer One 14 Z8-415 (UN.599SI.020) Laptop (Core...,Acer,One 14 Z8-415,34990.0,Medium-Low,64.0,NaN,Windows 11 Home Basic,512 GB,DDR4,Intel Iris Xe,14 Inches (35.56 cm),1920 x 1080 Pixels,16 GB,16:9,3.7,2 Cell,Li-Ion,45 W,1.49 Kg weight (Light-weight),No,Pure Silver,Full HD 1920 x 1080 Resolution Antiglare LED B...,0
1,AMD Quad Core Ryzen 3 - 7320U,AMD,Quad Core,Ryzen 3,Unknown,HP 15-fc0026AU (7L030PA) Laptop (AMD Quad Core...,HP,15-fc0026AU,36990.0,Medium-Low,NaN,2.4 Ghz,Windows 11 Home Basic,512 GB,LPDDR5,AMD Radeon,15.6 Inches (39.62 cm),1920 x 1080 Pixels,8 GB,16:9,NaN,3 Cell,Li-Ion,NaN,1.75 Kg weight,No,Natural Silver,FHD\nMicro-Edge\nAnti-Glare\nBrightness: 250 N...,0
2,Intel Core i3-1215U (12th Gen),Intel,Unknown,Core i3,12th Gen,HP 15s-fq5007TU (67V50PA) Laptop (Core i3 12th...,HP,15s-fq5007TU,37166.0,Medium-Low,66.0,4.2 Ghz,Windows 11 Home Basic,512 GB,DDR4,Intel UHD,15.6 Inches (39.62 cm),1920 x 1080 Pixels,8 GB,NaN,4.2,3 Cell,Li-Ion,65 W,1.69 Kg weight,No,Natural Silver,Diagonal\nFHD (1920 x 1080)\nIPS\nMicro-Edge\n...,0
3,Intel Core i7-13700HX (13th Gen),Intel,Unknown,Core i7,13th Gen,Acer Predator Helios Neo 16 PHN16-71 (NH.QLTSI...,Acer,Predator Helios Neo 16 PHN16-71,112980.0,Medium-High,61.0,NaN,Windows 11 Home Basic,1 TB,DDR5,NVIDIA GeForce RTX 4050,16 Inches (40.64 cm),1920 x 1200 Pixels,16 GB,NaN,4.4,4 Cell,Li-Ion,230 W,2.6 Kg weight,No,Black,IPS Display With WUXGA 1920 x 1200\nHigh-Brigh...,0
4,Intel Core i5-11400H (11th Gen),Intel,Unknown,Core i5,11th Gen,Asus TUF Gaming F17 FX706HF-HX018W Laptop (Cor...,Asus,TUF Gaming F17 FX706HF,57990.0,Medium-Low,65.0,2.7 Ghz,Windows 11 Home Basic,512 GB,DDR4,NVIDIA GeForce RTX 2050,17.3 Inches (43.94 cm),1920 x 1080 Pixels,8 GB,16:9,4.3,3 Cell,Li-Ion,150 W,2.60 Kg weight,No,Graphite Black,FHD (1920 x 1080) 16:9\nValue IPS-level\nAnti-...,0


In [39]:
# Reordering columns to bring certain columns before the 'Processor' column as specified
desired_columns_first = ['Name', 'Brand', 'Model_Name', 'Price', 'Price_Range', 'Spec Score']
new_columns_order = desired_columns_first + ['Processor', 'Processor_Brand', 'Core Configuration', 'Series', 'Generation'] + [col for col in laptops_data_new.columns if col not in (desired_columns_first + ['Processor', 'Processor_Brand', 'Core Configuration', 'Series', 'Generation'])]

# Apply the new column order to the DataFrame
laptops_data_new = data[new_columns_order]

# Display the first few rows to confirm the reordering
laptops_data_new.head()


,Name,Brand,Model_Name,Price,Price_Range,Spec Score,Processor,Processor_Brand,Core Configuration,Series,Generation,Clock-speed,Operating System,SSD Capacity,RAM Type,Graphics Processor,Display Size,Display Resolution,Capacity,Aspect Ratio,Overall Rating,Battery Cell,Battery type,Power Supply,Weight,Touchscreen,Colour(s),Display Features,Outlier_Flag
0,Acer One 14 Z8-415 (UN.599SI.020) Laptop (Core...,Acer,One 14 Z8-415,34990.0,Medium-Low,64.0,Intel Core i5-1155G7 (11th Gen),Intel,Unknown,Core i5,11th Gen,NaN,Windows 11 Home Basic,512 GB,DDR4,Intel Iris Xe,14 Inches (35.56 cm),1920 x 1080 Pixels,16 GB,16:9,3.7,2 Cell,Li-Ion,45 W,1.49 Kg weight (Light-weight),No,Pure Silver,Full HD 1920 x 1080 Resolution Antiglare LED B...,0
1,HP 15-fc0026AU (7L030PA) Laptop (AMD Quad Core...,HP,15-fc0026AU,36990.0,Medium-Low,NaN,AMD Quad Core Ryzen 3 - 7320U,AMD,Quad Core,Ryzen 3,Unknown,2.4 Ghz,Windows 11 Home Basic,512 GB,LPDDR5,AMD Radeon,15.6 Inches (39.62 cm),1920 x 1080 Pixels,8 GB,16:9,NaN,3 Cell,Li-Ion,NaN,1.75 Kg weight,No,Natural Silver,FHD\nMicro-Edge\nAnti-Glare\nBrightness: 250 N...,0
2,HP 15s-fq5007TU (67V50PA) Laptop (Core i3 12th...,HP,15s-fq5007TU,37166.0,Medium-Low,66.0,Intel Core i3-1215U (12th Gen),Intel,Unknown,Core i3,12th Gen,4.2 Ghz,Windows 11 Home Basic,512 GB,DDR4,Intel UHD,15.6 Inches (39.62 cm),1920 x 1080 Pixels,8 GB,NaN,4.2,3 Cell,Li-Ion,65 W,1.69 Kg weight,No,Natural Silver,Diagonal\nFHD (1920 x 1080)\nIPS\nMicro-Edge\n...,0
3,Acer Predator Helios Neo 16 PHN16-71 (NH.QLTSI...,Acer,Predator Helios Neo 16 PHN16-71,112980.0,Medium-High,61.0,Intel Core i7-13700HX (13th Gen),Intel,Unknown,Core i7,13th Gen,NaN,Windows 11 Home Basic,1 TB,DDR5,NVIDIA GeForce RTX 4050,16 Inches (40.64 cm),1920 x 1200 Pixels,16 GB,NaN,4.4,4 Cell,Li-Ion,230 W,2.6 Kg weight,No,Black,IPS Display With WUXGA 1920 x 1200\nHigh-Brigh...,0
4,Asus TUF Gaming F17 FX706HF-HX018W Laptop (Cor...,Asus,TUF Gaming F17 FX706HF,57990.0,Medium-Low,65.0,Intel Core i5-11400H (11th Gen),Intel,Unknown,Core i5,11th Gen,2.7 Ghz,Windows 11 Home Basic,512 GB,DDR4,NVIDIA GeForce RTX 2050,17.3 Inches (43.94 cm),1920 x 1080 Pixels,8 GB,16:9,4.3,3 Cell,Li-Ion,150 W,2.60 Kg weight,No,Graphite Black,FHD (1920 x 1080) 16:9\nValue IPS-level\nAnti-...,0


In [52]:
df = laptops_data_new.copy()

In [53]:
df.head()

,Name,Brand,Model_Name,Price,Price_Range,Spec Score,Processor,Processor_Brand,Core Configuration,Series,Generation,Clock-speed,Operating System,SSD Capacity,RAM Type,Graphics Processor,Display Size,Display Resolution,Capacity,Aspect Ratio,Overall Rating,Battery Cell,Battery type,Power Supply,Weight,Touchscreen,Colour(s),Display Features,Outlier_Flag
0,Acer One 14 Z8-415 (UN.599SI.020) Laptop (Core...,Acer,One 14 Z8-415,34990.0,Medium-Low,64.0,Intel Core i5-1155G7 (11th Gen),Intel,Unknown,Core i5,11th Gen,NaN,Windows 11 Home Basic,512 GB,DDR4,Intel Iris Xe,14 Inches (35.56 cm),1920 x 1080 Pixels,16 GB,16:9,3.7,2 Cell,Li-Ion,45 W,1.49 Kg weight (Light-weight),No,Pure Silver,Full HD 1920 x 1080 Resolution Antiglare LED B...,0
1,HP 15-fc0026AU (7L030PA) Laptop (AMD Quad Core...,HP,15-fc0026AU,36990.0,Medium-Low,NaN,AMD Quad Core Ryzen 3 - 7320U,AMD,Quad Core,Ryzen 3,Unknown,2.4 Ghz,Windows 11 Home Basic,512 GB,LPDDR5,AMD Radeon,15.6 Inches (39.62 cm),1920 x 1080 Pixels,8 GB,16:9,NaN,3 Cell,Li-Ion,NaN,1.75 Kg weight,No,Natural Silver,FHD\nMicro-Edge\nAnti-Glare\nBrightness: 250 N...,0
2,HP 15s-fq5007TU (67V50PA) Laptop (Core i3 12th...,HP,15s-fq5007TU,37166.0,Medium-Low,66.0,Intel Core i3-1215U (12th Gen),Intel,Unknown,Core i3,12th Gen,4.2 Ghz,Windows 11 Home Basic,512 GB,DDR4,Intel UHD,15.6 Inches (39.62 cm),1920 x 1080 Pixels,8 GB,NaN,4.2,3 Cell,Li-Ion,65 W,1.69 Kg weight,No,Natural Silver,Diagonal\nFHD (1920 x 1080)\nIPS\nMicro-Edge\n...,0
3,Acer Predator Helios Neo 16 PHN16-71 (NH.QLTSI...,Acer,Predator Helios Neo 16 PHN16-71,112980.0,Medium-High,61.0,Intel Core i7-13700HX (13th Gen),Intel,Unknown,Core i7,13th Gen,NaN,Windows 11 Home Basic,1 TB,DDR5,NVIDIA GeForce RTX 4050,16 Inches (40.64 cm),1920 x 1200 Pixels,16 GB,NaN,4.4,4 Cell,Li-Ion,230 W,2.6 Kg weight,No,Black,IPS Display With WUXGA 1920 x 1200\nHigh-Brigh...,0
4,Asus TUF Gaming F17 FX706HF-HX018W Laptop (Cor...,Asus,TUF Gaming F17 FX706HF,57990.0,Medium-Low,65.0,Intel Core i5-11400H (11th Gen),Intel,Unknown,Core i5,11th Gen,2.7 Ghz,Windows 11 Home Basic,512 GB,DDR4,NVIDIA GeForce RTX 2050,17.3 Inches (43.94 cm),1920 x 1080 Pixels,8 GB,16:9,4.3,3 Cell,Li-Ion,150 W,2.60 Kg weight,No,Graphite Black,FHD (1920 x 1080) 16:9\nValue IPS-level\nAnti-...,0


In [54]:
# Correct the casing for 'Quad Core' in the 'Core Configuration' column

# Display current unique values in 'Core Configuration'
df['Core Configuration'] = df['Core Configuration'].replace('Quad core', 'Quad Core')

# Verify the change
core_configuration_unique_corrected = df['Core Configuration'].unique()
core_configuration_unique_corrected


array(['Unknown', 'Quad Core', 'Dual Core', 'Hexa Core', 'Octa Core'],
      dtype=object)

Filling the null values in the "Clock-speed" column is indeed important, and using the mean or dropping the rows might not be the best approach. Here are

some robust strategies you can consider:

1. Imputation Using Similar Records:
Approach: Find records with similar specifications (e.g., same processor, series, generation, and other related attributes) and use their clock speed values to impute the missing ones.

Advantage: Ensures the imputed values are realistic and consistent with similar records.
2. Predictive Modeling:

Approach: Use machine learning models to predict the clock speed based on other features of the dataset. Features like processor, generation, and other related specifications can be used to train the model.

Advantage: Leverages the relationships between various attributes to make informed predictions.
3. Manual Imputation with Expert Knowledge:

Approach: Research and manually impute the missing values based on expert knowledge or reliable sources. This might be feasible if you have a manageable number of missing values or access to resources with the required information.
Advantage: Ensures high accuracy but might be time-consuming.

4. Combining Multiple Approaches:

Approach: Use a combination of imputation strategies. For example, start with predictive modeling to get a baseline imputation, and then refine using similar records or expert knowledge.
Advantage: Balances automation with accuracy and realism.

Given that the "Clock-speed" column is important and has 157 missing values, using predictive modeling combined with imputation using similar records would be a robust and efficient approach.

In [55]:
df['Clock-speed'].isnull().sum()

157

In [56]:
df.columns

Index(['Name', 'Brand', 'Model_Name', 'Price', 'Price_Range', 'Spec Score',
       'Processor', 'Processor_Brand', 'Core Configuration', 'Series',
       'Generation', 'Clock-speed', 'Operating System', 'SSD Capacity',
       'RAM Type', 'Graphics Processor', 'Display Size', 'Display Resolution',
       'Capacity', 'Aspect Ratio', 'Overall Rating', 'Battery Cell',
       'Battery type', 'Power Supply', 'Weight', 'Touchscreen', 'Colour(s)',
       'Display Features', 'Outlier_Flag'],
      dtype='object')

In [57]:
# Define a function to fill missing clock-speed values using similar records
def impute_clock_speed(data):
    for index, row in df.iterrows():
        if pd.isnull(row['Clock-speed']):
            # Find similar records based on Processor, Series, Generation, Model_Name, and Brand
            similar_records = df[
                (df['Processor'] == row['Processor']) &
                (df['Series'] == row['Series']) &
                (df['Generation'] == row['Generation']) &
                (df['Model_Name'] == row['Model_Name']) &
                (df['Brand'] == row['Brand']) &
                (df['Clock-speed'].notnull())
            ]

            if not similar_records.empty:
                # Impute with the most frequent clock speed value among similar records
                most_frequent_clock_speed = similar_records['Clock-speed'].mode()[0]
                df.at[index, 'Clock-speed'] = most_frequent_clock_speed
            else:
                # If no similar records found, leave it as NaN for now
                continue

# Perform the imputation
impute_clock_speed(df)

# Check the count of null values in the "Clock-speed" column after imputation
null_values_after_imputation = df['Clock-speed'].isnull().sum()
null_values_after_imputation


113

In [58]:
data = df.copy()

In [59]:
data['Clock-speed'].isnull().sum()

113

Random Forest model :

Summary of the Imputation Process
Data Preparation

Select Columns: Relevant columns (Clock-speed, Processor, Processor_Brand, Price, and Name) were selected.

Normalize Clock-speed: Cleaned the Clock-speed column by removing text like "Ghz" or "GHz" and converting the remaining values to numeric.

Encode Categorical Variables: Encoded the Processor, Processor_Brand, and Name columns using LabelEncoder.

Scale Numerical Variables: Standardized the Price column using StandardScaler.
Hyperparameter Tuning

Parameter Grid: A grid of hyperparameters was defined for the Random Forest model.

Grid Search: GridSearchCV was used to find the best hyperparameters.
Best Parameters:
max_depth: 30
max_features: 'sqrt'
min_samples_leaf: 1
min_samples_split: 10
n_estimators: 200

Model Training and Evaluation

Train Model: The Random Forest model was trained using the best parameters.
Mean Squared Error: The MSE on the training set was calculated to be 0.336.
Imputation

Predict Missing Values: The trained model was used to predict and fill missing Clock-speed values.

Update Dataset: The dataset was updated with the imputed Clock-speed values.
Save Updated Dataset

Reverse Transformations: Categorical variables were decoded and the scaling of the Price column was reversed.

Save Dataset: The updated dataset was saved to a new CSV file.

In [48]:
# Select relevant columns
selected_columns = ['Clock-speed', 'Processor', 'Processor_Brand', 'Price', 'Name']
data_selected = data[selected_columns].copy()

# Normalize the Clock-speed column
data_selected['Clock-speed'] = data_selected['Clock-speed'].astype(str).str.replace('Ghz', '').str.replace('GHz', '').str.strip()
data_selected['Clock-speed'] = pd.to_numeric(data_selected['Clock-speed'], errors='coerce')

# Encode categorical variables
label_encoders = {}
for col in ['Processor', 'Processor_Brand', 'Name']:
    le = LabelEncoder()
    data_selected[col] = data_selected[col].astype(str)
    data_selected[col] = le.fit_transform(data_selected[col])
    label_encoders[col] = le

# Scale numerical variables
scaler = StandardScaler()
data_selected['Price'] = scaler.fit_transform(data_selected[['Price']])

# Separate the data into training and testing sets
train_data = data_selected[data_selected['Clock-speed'].notnull()]
test_data = data_selected[data_selected['Clock-speed'].isnull()]

# Verify that train_data and test_data are not empty
if train_data.empty:
    raise ValueError("Training data is empty after filtering non-null Clock-speed values. Check the dataset for issues.")
if test_data.empty:
    raise ValueError("Test data is empty after filtering null Clock-speed values. Check the dataset for issues.")

# Split training data into features and target
X_train = train_data.drop(columns='Clock-speed')
y_train = train_data['Clock-speed']

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           n_jobs=-1,
                           verbose=2)

grid_search.fit(X_train, y_train)

# Best parameters from GridSearchCV
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the Random Forest model with the best parameters
rf = RandomForestRegressor(**best_params, random_state=42)
rf.fit(X_train, y_train)

# Calculate the MSE on the training set
y_train_pred = rf.predict(X_train)
mse = mean_squared_error(y_train, y_train_pred)
print(f"Mean Squared Error on the training set: {mse}")

# Predict missing values
X_test = test_data.drop(columns='Clock-speed')
test_data.loc[:, 'Clock-speed'] = rf.predict(X_test)

# Round the predicted Clock-speed values to 2 decimal places
test_data['Clock-speed'] = test_data['Clock-speed'].round(2)

# Combine the data back together
data_selected.loc[data_selected['Clock-speed'].isnull(), 'Clock-speed'] = test_data['Clock-speed']

# Reverse the scaling and encoding
data_selected['Price'] = scaler.inverse_transform(data_selected[['Price']])
for col in ['Processor', 'Processor_Brand', 'Name']:
    le = label_encoders[col]
    data_selected[col] = le.inverse_transform(data_selected[col])

# Update the original dataset with the imputed values, rounding to 2 decimal places
data['Clock-speed'] = data_selected['Clock-speed'].round(2)

# Save the updated dataset
output_file_path = 'updated_laptops_data_imputed_rf.csv'
data.to_csv(output_file_path, index=False)

print(f"Updated dataset saved to {output_file_path}")



Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best parameters found:  {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Mean Squared Error on the training set: 0.42583250696003017
Updated dataset saved to updated_laptops_data_imputed_rf.csv


<ipython-input-48-f1554ab911be>:70: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Steps for Further Hyperparameter Tuning
Expand the Parameter Grid:

Include more values for each hyperparameter to explore a broader range.
Use RandomizedSearchCV to sample from a larger parameter space if computational resources are a concern.
Train the Model with the Best Hyperparameters:

Retrain the model using the best hyperparameters found from the search.
Evaluate the model's performance and calculate the MSE.

In [49]:

# Select relevant columns
selected_columns = ['Clock-speed', 'Processor', 'Processor_Brand', 'Price', 'Name']
data_selected = df[selected_columns].copy()

# Normalize the Clock-speed column
data_selected['Clock-speed'] = data_selected['Clock-speed'].astype(str).str.replace('Ghz', '').str.replace('GHz', '').str.strip()
data_selected['Clock-speed'] = pd.to_numeric(data_selected['Clock-speed'], errors='coerce')

# Encode categorical variables
label_encoders = {}
for col in ['Processor', 'Processor_Brand', 'Name']:
    le = LabelEncoder()
    data_selected[col] = data_selected[col].astype(str)
    data_selected[col] = le.fit_transform(data_selected[col])
    label_encoders[col] = le

# Scale numerical variables
scaler = StandardScaler()
data_selected['Price'] = scaler.fit_transform(data_selected[['Price']])

# Separate the data into training and testing sets
train_data = data_selected[data_selected['Clock-speed'].notnull()]
test_data = data_selected[data_selected['Clock-speed'].isnull()]

# Verify that train_data and test_data are not empty
if train_data.empty:
    raise ValueError("Training data is empty after filtering non-null Clock-speed values. Check the dataset for issues.")
if test_data.empty:
    raise ValueError("Test data is empty after filtering null Clock-speed values. Check the dataset for issues.")

# Split training data into features and target
X_train = train_data.drop(columns='Clock-speed')
y_train = train_data['Clock-speed']

# Expanded hyperparameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True, False]
}

# Randomized search with 100 iterations
random_search = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=42),
                                   param_distributions=param_distributions,
                                   n_iter=100,
                                   cv=5,
                                   n_jobs=-1,
                                   verbose=2,
                                   random_state=42)

random_search.fit(X_train, y_train)

# Best parameters from RandomizedSearchCV
best_params = random_search.best_params_
print("Best parameters found: ", best_params)

# Train the Random Forest model with the best parameters
rf = RandomForestRegressor(**best_params, random_state=42)
rf.fit(X_train, y_train)

# Calculate the MSE on the training set
y_train_pred = rf.predict(X_train)
mse = mean_squared_error(y_train, y_train_pred)
print(f"Mean Squared Error on the training set: {mse}")

# Predict missing values
X_test = test_data.drop(columns='Clock-speed')
test_data.loc[:, 'Clock-speed'] = rf.predict(X_test)

# Round the predicted Clock-speed values to 2 decimal places
test_data.loc[:, 'Clock-speed'] = test_data['Clock-speed'].round(2)

# Combine the data back together
data_selected.loc[data_selected['Clock-speed'].isnull(), 'Clock-speed'] = test_data['Clock-speed']

# Reverse the scaling and encoding
data_selected['Price'] = scaler.inverse_transform(data_selected[['Price']])
for col in ['Processor', 'Processor_Brand', 'Name']:
    le = label_encoders[col]
    data_selected[col] = le.inverse_transform(data_selected[col])

# Update the original dataset with the imputed values, rounding to 2 decimal places
df.loc[data_selected.index, 'Clock-speed'] = data_selected['Clock-speed'].round(2)

# Save the updated dataset
output_file_path = 'updated_laptops_data_imputed_rsv.csv'
df.to_csv(output_file_path, index=False)

print(f"Updated dataset saved to {output_file_path}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters found:  {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 10, 'bootstrap': True}
Mean Squared Error on the training set: 0.4220535090075249
Updated dataset saved to updated_laptops_data_imputed_rsv.csv


Initialize RandomizedSearchCV:


Set the estimator (e.g., RandomForestRegressor).
Provide the parameter distributions.

Define the number of iterations (n_iter), cross-validation strategy (cv), number of jobs (n_jobs), verbosity (verbose), and random state (random_state).


RandomizedSearchCV is an efficient method for hyperparameter tuning, especially when dealing with large parameter spaces. It allows for a more comprehensive exploration of hyperparameters while keeping computation time manageable.

By following these steps, you can optimize your model and improve its performance for tasks such as imputing missing values.

In [65]:
# Normalize the Clock-speed column
data_selected['Clock-speed'] = data_selected['Clock-speed'].astype(str).str.replace('Ghz', '').str.replace('GHz', '').str.strip()
data_selected['Clock-speed'] = pd.to_numeric(data_selected['Clock-speed'], errors='coerce')

# Encode categorical variables
label_encoders = {}
for col in ['Processor', 'Processor_Brand', 'Name']:
    le = LabelEncoder()
    data_selected[col] = data_selected[col].astype(str)
    data_selected[col] = le.fit_transform(data_selected[col])
    label_encoders[col] = le

# Scale numerical variables
scaler = StandardScaler()
data_selected['Price'] = scaler.fit_transform(data_selected[['Price']])

# Separate the data into training and testing sets based on the presence of null values in 'Clock-speed'
train_data = data_selected[data_selected['Clock-speed'].notnull()]
test_data = data_selected[data_selected['Clock-speed'].isnull()]

# Verify that train_data and test_data are not empty
if train_data.empty:
    raise ValueError("Training data is empty after filtering non-null Clock-speed values. Check the dataset for issues.")
if test_data.empty:
    raise ValueError("Test data is empty after filtering null Clock-speed values. Check the dataset for issues.")

# Split training data into features and target
X_train = train_data.drop(columns='Clock-speed')
y_train = train_data['Clock-speed']
X_test = test_data.drop(columns='Clock-speed')

# 1. KNN Imputation
knn_imputer = KNNImputer(n_neighbors=5)
data_knn_imputed = knn_imputer.fit_transform(data_selected)
data_selected['Clock-speed_knn'] = data_knn_imputed[:, 0]  # Assuming 'Clock-speed' is the first column

# 2. Random Forest Imputation
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
data_selected.loc[test_data.index, 'Clock-speed_rf'] = rf.predict(X_test)

# 3. XGBoost Imputation
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
data_selected.loc[test_data.index, 'Clock-speed_xgb'] = xgb_model.predict(X_test)

# Compare performance using the training data
print(f"KNN Imputer MSE: {mean_squared_error(y_train, knn_imputer.fit_transform(train_data)[:, 0])}")
print(f"Random Forest Imputer MSE: {mean_squared_error(y_train, rf.predict(X_train))}")
print(f"XGBoost Imputer MSE: {mean_squared_error(y_train, xgb_model.predict(X_train))}")



KNN Imputer MSE: 0.0
Random Forest Imputer MSE: 0.11676174491392839
XGBoost Imputer MSE: 0.16621353334975025


In [ ]:
# Select relevant columns
selected_columns = ['Clock-speed', 'Processor', 'Processor_Brand', 'Price', 'Name']
data_selected = data[selected_columns].copy()

# Normalize the Clock-speed column
data_selected['Clock-speed'] = data_selected['Clock-speed'].astype(str).str.replace('Ghz', '').str.replace('GHz', '').str.strip()
data_selected['Clock-speed'] = pd.to_numeric(data_selected['Clock-speed'], errors='coerce')

# Encode categorical variables
label_encoders = {}
for col in ['Processor', 'Processor_Brand', 'Name']:
    le = LabelEncoder()
    data_selected[col] = data_selected[col].astype(str)
    data_selected[col] = le.fit_transform(data_selected[col])
    label_encoders[col] = le

# Scale numerical variables
scaler = StandardScaler()
data_selected['Price'] = scaler.fit_transform(data_selected[['Price']])

# Separate the data into training and testing sets based on the presence of null values in 'Clock-speed'
train_data = data_selected[data_selected['Clock-speed'].notnull()]
test_data = data_selected[data_selected['Clock-speed'].isnull()]

# Verify that train_data and test_data are not empty
if train_data.empty:
    raise ValueError("Training data is empty after filtering non-null Clock-speed values. Check the dataset for issues.")
if test_data.empty:
    raise ValueError("Test data is empty after filtering null Clock-speed values. Check the dataset for issues.")

# Split training data into features and target
X_train = train_data.drop(columns='Clock-speed')
y_train = train_data['Clock-speed']
X_test = test_data.drop(columns='Clock-speed')

# 1. KNN Imputation
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
data_knn_imputed = knn_imputer.fit_transform(data_selected)
data_selected['Clock-speed_knn'] = data_knn_imputed[:, 0]  # Assuming 'Clock-speed' is the first column

# 2. Random Forest Imputation
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
data_selected.loc[test_data.index, 'Clock-speed_rf'] = rf.predict(X_test)

# Cross-validation for Random Forest
rf_cv_scores = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rf_cv_mse = -rf_cv_scores.mean()
print(f"Random Forest Cross-Validated MSE: {rf_cv_mse}")

# Hyperparameter tuning for Random Forest using GridSearchCV
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search_rf.fit(X_train, y_train)
best_params_rf = grid_search_rf.best_params_
print(f"Best parameters for Random Forest: {best_params_rf}")

# Train the Random Forest model with the best parameters
rf_best = RandomForestRegressor(**best_params_rf, random_state=42)
rf_best.fit(X_train, y_train)
data_selected.loc[test_data.index, 'Clock-speed_rf_best'] = rf_best.predict(X_test)

# 3. XGBoost Imputation
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
data_selected.loc[test_data.index, 'Clock-speed_xgb'] = xgb_model.predict(X_test)

# Cross-validation for XGBoost
xgb_cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
xgb_cv_mse = -xgb_cv_scores.mean()
print(f"XGBoost Cross-Validated MSE: {xgb_cv_mse}")

# Hyperparameter tuning for XGBoost using GridSearchCV
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search_xgb.fit(X_train, y_train)
best_params_xgb = grid_search_xgb.best_params_
print(f"Best parameters for XGBoost: {best_params_xgb}")

# Train the XGBoost model with the best parameters
xgb_best = xgb.XGBRegressor(**best_params_xgb, objective='reg:squarederror', random_state=42)
xgb_best.fit(X_train, y_train)
data_selected.loc[test_data.index, 'Clock-speed_xgb_best'] = xgb_best.predict(X_test)

# Compare performance using the training data
print(f"KNN Imputer MSE: {mean_squared_error(y_train, knn_imputer.fit_transform(train_data)[:, 0])}")
print(f"Random Forest Best Imputer MSE: {mean_squared_error(y_train, rf_best.predict(X_train))}")
print(f"XGBoost Best Imputer MSE: {mean_squared_error(y_train, xgb_best.predict(X_train))}")


Random Forest Cross-Validated MSE: 0.9338166873173005
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best parameters for Random Forest: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
XGBoost Cross-Validated MSE: 0.9746854107045019
Fitting 5 folds for each of 405 candidates, totalling 2025 fits
Best parameters for XGBoost: {'colsample_bytree': 0.6, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.8}
KNN Imputer MSE: 0.0
Random Forest Best Imputer MSE: 0.42583250696003017
XGBoost Best Imputer MSE: 0.4827984903270324
